<a href="https://colab.research.google.com/github/RamkrishnaKamble/CL465/blob/main/465_project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

In [ ]:
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=300)
def import_stock_data(tickers, start = startDate):
    data = pd.DataFrame()
    if len([tickers]) ==1:
        data[tickers] = wb.DataReader(tickers, data_source='yahoo', start = start)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = wb.DataReader(t, data_source='yahoo', start = start)['Adj Close']
    return(data)

In [ ]:
#Single Simulation
plt.style.use('fivethirtyeight')
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=100)
data1 = import_stock_data("googl", start = startDate)
log_returns_s = (np.log(1+data1.pct_change()))
drift = log_returns_s.mean() - (0.5*log_returns_s.var())
dr_s = np.exp(drift +  (log_returns_s.std()*np.random.normal(loc = 0, scale = np.sqrt(1/100) * log_returns_s.std())))
pred_prices = dr_s*data1
pred_prices.plot()
plt.ylabel('price')
plt.axhline(y=float( data1.iloc[-1]), color='k', linestyle='dashed',linewidth=2,label = 'Current Value: ' + str(round(float( data1.iloc[-1]),3)))

In [ ]:
def log_returns(data):
    return (np.log(1+data.pct_change()))

In [ ]:
def drift_calc(data):
    lr = log_returns(data)
    u = lr.mean()
    var = lr.var()
    drift = u-(0.5*var)
    try:
        return drift.values
    except:
        return drift

In [ ]:
def daily_returns(data, days, iterations):
    ft = drift_calc(data)
    try:
        stv = log_returns(data).std().values
    except:
        stv = log_returns(data).std()
    dr = np.exp(ft + stv * norm.ppf(np.random.rand(days, iterations)))
    return dr

In [ ]:
def probs_find(predicted, higherthan, on = 'value'):
    if on == 'return':
        predicted0 = predicted.iloc[0,0]
        predicted = predicted.iloc[-1]
        predList = list(predicted)
        over = [(i*100)/predicted0 for i in predList if ((i-predicted0)*100)/predicted0 >= higherthan]
        less = [(i*100)/predicted0 for i in predList if ((i-predicted0)*100)/predicted0 < higherthan]
    elif on == 'value':
        predicted = predicted.iloc[-1]
        predList = list(predicted)
        over = [i for i in predList if i >= higherthan]
        less = [i for i in predList if i < higherthan]
    else:
        print("'on' must be either value or return")
    return (len(over)/(len(over)+len(less)))

In [ ]:
def simulate_mc(data, days, iterations, plot=True):
    # Generate daily returns
    returns = daily_returns(data, days, iterations)
    # Create empty matrix
    price_list = np.zeros_like(returns)
    # Put the last actual price in the first row of matrix. 
    price_list[0] = data.iloc[-1]
    # Calculate the price of each day
    for t in range(1,days):
        price_list[t] = price_list[t-1]*returns[t]
    #Storing Value of simulations
    price_simulations_t = pd.DataFrame(price_list)
    # Plot Option
    if plot == True:
        x = pd.DataFrame(price_list).iloc[-1]
        sns.distplot(x)
        plt.xlabel("Stock Price")
        plt.show()
    # Graphing simulations in sequence
    plt.figure(figsize=(10,5))
    plt.plot(price_simulations_t,linestyle='-',linewidth=1)
    plt.axhline(y=float( data.iloc[-1]), color='k', linestyle='dashed',linewidth=2,label = 'Current Value: ' + str(round(float( data.iloc[-1]),3)))
    
    title = " Monte Carlo Simulations"
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.xticks(rotation=45)
    plt.legend(loc="upper right")    
    plt.show()
    # Printing information about stock
    try:
        [print(nam) for nam in data.columns]
    except:
        print(data.name)
    print(f"Days: {days-1}")
    print(f"Expected Value: ${round(pd.DataFrame(price_list).iloc[-1].mean(),2)}")
    print(f"Return: {round(100*(pd.DataFrame(price_list).iloc[-1].mean()-data.iloc[-1])/pd.DataFrame(price_list).iloc[-1].mean(),2)}%")
    print(f"Probability of Breakeven: {probs_find(pd.DataFrame(price_list),0, on='return')}")
   
          
    return pd.DataFrame(price_list)

In [ ]:
def monte_carlo(tickers, days_forecast, iterations, start_date = '2000-1-1', plotten=False):
    data = import_stock_data(tickers, start=start_date)
    simulatedDF = []
    for t in range(len(tickers)):
        y = simulate_mc(data.iloc[:,t], (days_forecast+1), iterations)
        if plotten == True:
            forplot = y.iloc[:,0:10]
            forplot.plot(figsize=(15,4))
        y['ticker'] = tickers[t]
        cols = y.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        y = y[cols]
        simulatedDF.append(y)
    simulatedDF = pd.concat(simulatedDF)
    return simulatedDF

In [ ]:
ret_sim_df = monte_carlo(['GOOG','FB','AAPL'], 100, 10000,  start_date= startDate)